다대다 RNN은 대표적으로 품사 태깅, 개체명 인식 등에서 사용

# 

# *Char RNN*

RNN 입출력 단위가 단어 레벨(word-level)이 아니라 문자 레벨(character-level)로 하여 RNN을 구현한다면, 이를 <b>문자 단위 RNN</b>라고 함.<br>
<u>RNN 구조 자체가 달라진 것은 아니고, 입,출력의 단위가 문자로 바뀜</u><br><br>
<b>문자 단위 RNN을 다대다 구조</b>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

### *processing*

문자 시퀀스 apple을 입력받아 pple!를 출력하는 RNN 구현

In [2]:
input_str = 'apple'
label_str = 'pple!'

In [3]:
char_vocab = sorted(list(set(input_str + label_str)))

In [4]:
vocab_size = len(char_vocab)
print ('문자 집합의 크기 : {}'.format(vocab_size))

문자 집합의 크기 : 5


# 

# *Encoding*

In [5]:
char_to_index = {v: k for k, v in enumerate(char_vocab)}
index_to_char = {k: v for k, v in enumerate(char_vocab)}
print(f"char_to_index : {char_to_index}")
print(f"index_to_char : {index_to_char}")

char_to_index : {'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}
index_to_char : {0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [6]:
x_data = [char_to_index[i] for i in input_str]
y_data = [char_to_index[i] for i in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


### *batch 차원 추가*

In [7]:
x_data = torch.LongTensor(x_data)
x_data = F.one_hot(x_data, num_classes=vocab_size) # one-hot encoding (LongTensor 필요)
X = x_data.unsqueeze(0).type(torch.FloatTensor)
Y = torch.LongTensor(y_data).unsqueeze(0)

In [8]:
print(X)
print(Y)

tensor([[[0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0.]]])
tensor([[4, 4, 3, 2, 0]])


In [9]:
print(f"훈련 데이터 크기 : {X.shape}")
print(f"레이블 크기 : {Y.shape}")

훈련 데이터 크기 : torch.Size([1, 5, 5])
레이블 크기 : torch.Size([1, 5])


# 

# *Train*

In [10]:
input_size = vocab_size
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [11]:
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True) # RNN 셀 구현
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현

    def forward(self, x): # 구현한 RNN 셀과 출력층을 연결
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [12]:
net = Net(input_size, hidden_size, output_size)
print(net)

Net(
  (rnn): RNN(5, 5, batch_first=True)
  (fc): Linear(in_features=5, out_features=5, bias=True)
)


In [16]:
outputs = net(X)
print(outputs)
print(outputs.shape)

tensor([[[-0.2175, -0.2541,  0.5675, -0.0843,  0.1576],
         [-0.0720, -0.2235,  0.3873, -0.4740,  0.5187],
         [-0.1215, -0.1076,  0.4717, -0.3196,  0.3633],
         [-0.3617, -0.2319,  0.7496, -0.3049,  0.4235],
         [ 0.0344,  0.1267,  0.7013, -0.5676,  0.5601]]],
       grad_fn=<AddBackward0>)
torch.Size([1, 5, 5])


In [17]:
print(outputs.view(-1, input_size)) # 2차원 텐서로 변환
print(outputs.view(-1, input_size).shape) # 2차원 텐서로 변환

tensor([[-0.2175, -0.2541,  0.5675, -0.0843,  0.1576],
        [-0.0720, -0.2235,  0.3873, -0.4740,  0.5187],
        [-0.1215, -0.1076,  0.4717, -0.3196,  0.3633],
        [-0.3617, -0.2319,  0.7496, -0.3049,  0.4235],
        [ 0.0344,  0.1267,  0.7013, -0.5676,  0.5601]], grad_fn=<ViewBackward>)
torch.Size([5, 5])


In [18]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [19]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1)) # view를 하는 이유는 Batch 차원 제거를 위해
    loss.backward() # 기울기 계산
    optimizer.step() # 아까 optimizer 선언 시 넣어둔 파라미터 업데이트

    # 아래 세 줄은 모델이 실제 어떻게 예측했는지를 확인하기 위한 코드.
    result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.52280592918396 prediction:  [[2 4 2 2 2]] true Y:  [4, 4, 3, 2, 0] prediction str:  epeee
1 loss:  1.2713689804077148 prediction:  [[4 4 4 4 4]] true Y:  [4, 4, 3, 2, 0] prediction str:  ppppp
2 loss:  1.073270559310913 prediction:  [[4 4 3 4 0]] true Y:  [4, 4, 3, 2, 0] prediction str:  pplp!
3 loss:  0.8618305325508118 prediction:  [[4 4 3 4 0]] true Y:  [4, 4, 3, 2, 0] prediction str:  pplp!
4 loss:  0.6540454030036926 prediction:  [[4 4 3 2 0]] true Y:  [4, 4, 3, 2, 0] prediction str:  pple!
5 loss:  0.47488659620285034 prediction:  [[4 4 3 2 0]] true Y:  [4, 4, 3, 2, 0] prediction str:  pple!
6 loss:  0.34092622995376587 prediction:  [[4 4 3 2 0]] true Y:  [4, 4, 3, 2, 0] prediction str:  pple!
7 loss:  0.2342413365840912 prediction:  [[4 4 3 2 0]] true Y:  [4, 4, 3, 2, 0] prediction str:  pple!
8 loss:  0.15783968567848206 prediction:  [[4 4 3 2 0]] true Y:  [4, 4, 3, 2, 0] prediction str:  pple!
9 loss:  0.10820503532886505 prediction:  [[4 4 3 2 0]] true Y:  [4, 4, 3

# 

# 

# *Char RNN(문장 단위 RNN)*

### *dictionary 정의*

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [2]:
sentence = """배를 만들고 싶다면 사람들을 모아 나무를 모으거나 작업과 일을 할당하지 말고 끝없이 광대한 바다를 동경하도록 가르치십시오."""

In [3]:
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}
print(char_dic)

{'시': 0, '도': 1, '경': 2, '대': 3, '치': 4, '할': 5, '가': 6, '나': 7, '하': 8, '배': 9, '르': 10, '이': 11, ' ': 12, '만': 13, '모': 14, '과': 15, '다': 16, '람': 17, '를': 18, '바': 19, '일': 20, '말': 21, '지': 22, '고': 23, '사': 24, '록': 25, '십': 26, '무': 27, '으': 28, '당': 29, '광': 30, '들': 31, '면': 32, '싶': 33, '없': 34, '을': 35, '업': 36, '오': 37, '작': 38, '한': 39, '동': 40, '.': 41, '거': 42, '아': 43, '끝': 44}


In [4]:
dic_size = len(char_dic)
print('문자 집합의 크기 : {}'.format(dic_size))

문자 집합의 크기 : 45


### *one-hot encoding*

In [5]:
# 하이퍼파라미터 설정
hidden_size = dic_size
sequence_length = 10  # 임의 숫자 지정
learning_rate = 0.1

In [6]:
# 데이터 구성
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str])  # x str to index
    y_data.append([char_dic[c] for c in y_str])  # y str to index

0 배를 만들고 싶다면 -> 를 만들고 싶다면 
1 를 만들고 싶다면  ->  만들고 싶다면 사
2  만들고 싶다면 사 -> 만들고 싶다면 사람
3 만들고 싶다면 사람 -> 들고 싶다면 사람들
4 들고 싶다면 사람들 -> 고 싶다면 사람들을
5 고 싶다면 사람들을 ->  싶다면 사람들을 
6  싶다면 사람들을  -> 싶다면 사람들을 모
7 싶다면 사람들을 모 -> 다면 사람들을 모아
8 다면 사람들을 모아 -> 면 사람들을 모아 
9 면 사람들을 모아  ->  사람들을 모아 나
10  사람들을 모아 나 -> 사람들을 모아 나무
11 사람들을 모아 나무 -> 람들을 모아 나무를
12 람들을 모아 나무를 -> 들을 모아 나무를 
13 들을 모아 나무를  -> 을 모아 나무를 모
14 을 모아 나무를 모 ->  모아 나무를 모으
15  모아 나무를 모으 -> 모아 나무를 모으거
16 모아 나무를 모으거 -> 아 나무를 모으거나
17 아 나무를 모으거나 ->  나무를 모으거나 
18  나무를 모으거나  -> 나무를 모으거나 작
19 나무를 모으거나 작 -> 무를 모으거나 작업
20 무를 모으거나 작업 -> 를 모으거나 작업과
21 를 모으거나 작업과 ->  모으거나 작업과 
22  모으거나 작업과  -> 모으거나 작업과 일
23 모으거나 작업과 일 -> 으거나 작업과 일을
24 으거나 작업과 일을 -> 거나 작업과 일을 
25 거나 작업과 일을  -> 나 작업과 일을 할
26 나 작업과 일을 할 ->  작업과 일을 할당
27  작업과 일을 할당 -> 작업과 일을 할당하
28 작업과 일을 할당하 -> 업과 일을 할당하지
29 업과 일을 할당하지 -> 과 일을 할당하지 
30 과 일을 할당하지  ->  일을 할당하지 말
31  일을 할당하지 말 -> 일을 할당하지 말고
32 일을 할당하지 말고 -> 을 할당하지 말고 
33 을 할당하지 말고  ->  할당하지 말고 끝
34  할당하지 말고 끝 -> 할당하지 말고 끝없
35 할당하지 말고 끝없 -> 당하지 말고 끝없이
36

In [7]:
print(x_data[0]) # 배를 만들고 싶다면
print(y_data[0]) # 를 만들고 싶다면 

[9, 18, 12, 13, 31, 23, 12, 33, 16, 32]
[18, 12, 13, 31, 23, 12, 33, 16, 32, 12]


In [8]:
# x_one_hot = [np.eye(dic_size)[x] for x in x_data] # x 데이터는 원-핫 인코딩
# X = torch.FloatTensor(x_one_hot)
# Y = torch.LongTensor(y_data)
x_data = F.one_hot(torch.LongTensor(x_data), num_classes=dic_size) # one-hot encoding (LongTensor 필요)
X = x_data.type(torch.FloatTensor)
Y = torch.LongTensor(y_data)

In [9]:
print(f"X shape : {X.shape}")
print(f"Y shape : {Y.shape}")
print(f"X : {X}")
print(f"Y : {Y}")

X shape : torch.Size([58, 10, 45])
Y shape : torch.Size([58, 10])
X : tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
       

In [10]:
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
      

In [11]:
print(Y[0])

tensor([18, 12, 13, 31, 23, 12, 33, 16, 32, 12])


### *train*

In [12]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers): # 현재 hidden_size는 dic_size와 같음.
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True) # num_layer : 은닉층 갯수
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [13]:
net = Net(dic_size, hidden_size, 2) # 이번에는 층을 두 개 쌓습니다.

In [14]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [15]:
outputs = net(X)

torch.Size([58, 10, 45])


In [17]:
print(outputs.shape) # 3차원 텐서
print(outputs)

torch.Size([58, 10, 45])
tensor([[[ 0.1784,  0.2366,  0.0113,  ..., -0.0373,  0.2178,  0.0005],
         [ 0.2868,  0.1274, -0.1131,  ..., -0.1049,  0.0895,  0.0255],
         [ 0.3012,  0.2528, -0.0762,  ..., -0.1812,  0.1059,  0.0378],
         ...,
         [ 0.3040,  0.2580, -0.1603,  ..., -0.1550,  0.2047,  0.1423],
         [ 0.3755,  0.2062, -0.1192,  ..., -0.1115,  0.1821,  0.0052],
         [ 0.3191,  0.2543, -0.1698,  ..., -0.1186,  0.1224,  0.0527]],

        [[ 0.2252,  0.1885, -0.0836,  ..., -0.0336,  0.1587,  0.0039],
         [ 0.2831,  0.2169, -0.0421,  ..., -0.1862,  0.1293,  0.0233],
         [ 0.3589,  0.2465, -0.0999,  ..., -0.0718,  0.1473,  0.0478],
         ...,
         [ 0.3768,  0.2076, -0.1190,  ..., -0.1110,  0.1802,  0.0051],
         [ 0.3170,  0.2533, -0.1695,  ..., -0.1195,  0.1216,  0.0524],
         [ 0.3813,  0.2951, -0.1023,  ..., -0.1989,  0.1630, -0.0227]],

        [[ 0.2444,  0.2313, -0.0337,  ..., -0.1034,  0.1502, -0.0589],
         [ 0.2828,  

In [18]:
print(outputs.view(-1, dic_size).shape) # 2차원 텐서로 변환.

torch.Size([580, 45])


In [19]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([58, 10])
torch.Size([580])


# 

In [20]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X) # (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    # results의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오지만
            predict_str += ''.join([char_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += char_set[result[-1]]

    print(predict_str)

도시대시시도시시시시시시시시시시시시대시시시시시시시시시시대시시대시대시대대시시시도시도대시대시시시시시시시시시도시시시대시시시시대시
                                                                   
를 모 을 가 을  당을당을 당    을 을  을 을당  을을 을 을 을  을당을  을고 을당  을당  을  당당 을  
                                                                   
  말대거 모대거  거거   모       거  거        거   거  거    거 말거  말거 모 거   거  모
면할광람도모작고도록록 나치도나람없없나 나과고나없들나나과들나 들고고 광들도나과들나 광과고 없록나광과들 할한도나광고고나나 광
면 가다면 대아 바아  작이 당일  이 일이 당아  일이 이 모이  일이 록다들  일이 모만  광아면 록아  일이    
를 싶다를 할다를 할다  아치  할다다  아를 모다  할람경  아를  아를 싶람를  경를 할다를 할다를 할과를 아거아과 
를 사다  모경하 모경하 모없  모없다  사하 모경를 싶없다  사다  록다 모없다  경하 모다하 모없하 모없하   없없 
를 사하  바없하 바고하   하 모고   사하  없을  없   사하  바하  없하  없하  대하 바없하  없하   없을 
를 바람   고들 바고들 바을고  고   고   을을  무을  무   바고  고업  고  바고들 바람들 바을업   고  
를 바대이 끝다들 바람들  무고  무업  나이  무이  무업  당업  록업  을업  다이 바대들 동다이  없업들  바이 
를 바대이 끝다를 동다를도 록다  당다  을를도 다으  록다  바다  록다  다이과끝다를 동다를 동다를  다다를과 당이 
를 사을면 광을면 당다를  일를  당이 광을  당을업면 당을  당를과 사다  다를 일다과  다면 사다를 록다하를  당를 
를 사다하 사경하 사경하  말고 할없람 모경  당없르면 당다  없하치 말고 끝다하 모경